In [ ]:
#old hourly cluster weight method
def cal_hourly_cluster_weight():
    global staypts_df
    global cluster_hourly_df
    global hourly_start_df
    global hourly_end_df

    #create two new dataframes with start and end of each hour for each cluster
    hourly_start_df = staypts_df.resample('H').first()
    hourly_end_df = staypts_df.resample('H').last()
    hourly_start_df = hourly_start_df.dropna()
    hourly_end_df = hourly_end_df.dropna()

    for i in range(0, 24):
        cluster_hourly_df['Date'] = 0
        cluster_hourly_df['ClusterId'] = 0
        cluster_hourly_df['AvgLat'] = 0
        cluster_hourly_df['AvgLon'] = 0
        cluster_hourly_df[i] = 0

    for i in range(0, len(hourly_start_df)):
    #for i in range(0, staypts_df['ClusterId'].nunique()):
        if hourly_end_df['Timestamp'][i] == hourly_start_df['Timestamp'][i]:
            k = (hourly_start_df.index[i] + pd.Timedelta(hours=1)) - hourly_start_df['Timestamp'][i]
        else:
            k = hourly_end_df['Timestamp'][i] - hourly_start_df['Timestamp'][i]

        l = (k / np.timedelta64(1, 'm')).astype(int)
        date_read = hourly_start_df.index[i].date()
        cluster_id = hourly_start_df['ClusterId'][i]
        ClusterMeanLat = hourly_start_df['ClusterMeanLat'][i]

        ClusterMeanLon = hourly_start_df['ClusterMeanLon'][i]

        col_name = hourly_start_df.index[i].hour 
        #import pdb; pdb.set_trace()
        cluster_hourly_df.loc[i, 'AvgLat'] = ClusterMeanLat
        cluster_hourly_df.loc[i, 'AvgLon'] = ClusterMeanLon
        cluster_hourly_df.loc[i, 'Date'] = date_read
        cluster_hourly_df.loc[i, 'ClusterId'] = cluster_id
        cluster_hourly_df.loc[i, col_name] = round((l)/60,4)

    cluster_hourly_df = cluster_hourly_df.fillna(0)
    cluster_hourly_df = cluster_hourly_df.groupby(['Date', 'ClusterId', 'AvgLat', 'AvgLon']).sum()
    cluster_hourly_df = cluster_hourly_df.reset_index(level=[0,1,2,3])